In [19]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle


### ввод
df = pd.read_csv('./dataset.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.drop('Post Promotion Status', axis=1, inplace=True)



### нормируем
def is_binary(x):
    return x.unique().shape[0] == 2

def normalize(x):
    return (x - x.mean()) / x.std()

bin_free = df.columns[~df.apply(is_binary)]
df[bin_free] = df[bin_free].apply(normalize, axis=0)
df['w0_reg_constant'] = 1



### перемешиваем
df = shuffle(df)


# функция градиентного спуска
# где new_w задает измение веса с помощью градиентного спуска
def gradient_descent(X, y, theta=1e-3, epsilon=1e-5):
    dw = np.inf
    w = np.random.normal(size=X.shape[1])
    N = X.shape[1]

    while dw >= epsilon:
        new_w = 2*theta/N*(X.T.dot(X.dot(w)) - X.T.dot(y))
        dw = np.linalg.norm(w - new_w)
        w = new_w

    return w

# остальные функции
def R2(x, y):
    return 1 - np.sum(np.power(y - x, 2)) / np.sum(np.power(y - y.mean(), 2))

def reg_prediction(X, w):
    return X.dot(w)


def RMSE(x, y):
    return np.sqrt(np.sum(np.power(y - x, 2)) / y.shape[0])



# делим на фолды и мутим свое грязное дело
# вторые копии для наглядости
folds_index = 5
fold_size = round(df.shape[0] / folds_index)

features = pd.DataFrame()
RMSE_test = []
RMSE_train = []
R2_test = []
R2_train = []

features2 = pd.DataFrame()
RMSE_test2 = []
RMSE_train2 = []
R2_test2 = []
R2_train2 = []

for i in range(folds_index):
    test = df[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = df[(i + 1) * fold_size:]
    else:
        train = df[:i * fold_size]
        if i != 4:
            train = train.append(df[(i + 1) * fold_size:], ignore_index=False)


    Features = train.drop('Target', axis=1)
    Target = train['Target']
    w = gradient_descent(Features, Target, 1e-3)
    features = features.append(w, ignore_index=True)
    w2 = gradient_descent(Features, Target, 1e-4)
    features2 = features2.append(w2, ignore_index=True)

    train_pred = reg_prediction(train.drop('Target', axis=1), w)
    R2_train.append(R2(train_pred, train['Target']))
    RMSE_train.append(RMSE(train_pred, train['Target']))

    test_pred = reg_prediction(test.drop('Target', axis=1), w)
    R2_test.append(R2(test_pred, test['Target']))
    RMSE_test.append(RMSE(test_pred, test['Target']))

    train_pred2 = reg_prediction(train.drop('Target', axis=1), w2)
    R2_train2.append(R2(train_pred2, train['Target']))
    RMSE_train2.append(RMSE(train_pred2, train['Target']))

    test_pred2 = reg_prediction(test.drop('Target', axis=1), w2)
    R2_test2.append(R2(test_pred2, test['Target']))
    RMSE_test2.append(RMSE(test_pred2, test['Target']))

# вывод результатов с шагом 0.001
res_df = pd.DataFrame(np.vstack([R2_test, R2_train, RMSE_test, RMSE_train]), index=['R2_test', 'R2_train', 'RMSE_test', 'RMSE_train'])
res_df = res_df.append(features.T)
res_df.columns = ['T1', 'T2', 'T3', 'T4', 'T5']
res_df = pd.concat([res_df, res_df.mean(axis=1).rename('E(mean)'), res_df.std(axis=1).rename('STD')], axis=1)

res_df

H:\programs\anaconda\lib\site-packages\pandas\core\nanops.py:468: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values)**2)


,T1,T2,T3,T4,T5,E(mean),STD
R2_test,1.0,1.0,1.000000e+00,1.000000e+00,1.0,1.000000e+00,0.000000
R2_train,1.0,1.0,1.000000e+00,1.000000e+00,1.0,1.000000e+00,0.000000
RMSE_test,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000
RMSE_train,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000
Base Time,NaN,NaN,9.159081e+302,9.959388e+302,NaN,9.559234e+302,inf
CC1,NaN,NaN,-inf,-inf,NaN,-inf,NaN
CC2,NaN,NaN,-inf,-inf,NaN,-inf,NaN
CC3,NaN,NaN,-inf,-inf,NaN,-inf,NaN
CC4,NaN,NaN,-inf,-inf,NaN,-inf,NaN
CC5,NaN,NaN,-4.706981e+303,-5.507164e+303,NaN,-5.107072e+303,inf


In [22]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle


### ввод
df = pd.read_csv('./dataset.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.drop('Post Promotion Status', axis=1, inplace=True)



### нормируем
def is_binary(x):
    return x.unique().shape[0] == 2

def normalize(x):
    return (x - x.mean()) / x.std()

bin_free = df.columns[~df.apply(is_binary)]
df[bin_free] = df[bin_free].apply(normalize, axis=0)
df['w0_reg_constant'] = 1



### перемешиваем
df = shuffle(df)


# функция градиентного спуска
# где new_w задает измение веса с помощью градиентного спуска
def gradient_descent(X, y, theta=1e-3, epsilon=1e-5):
    dw = np.inf
    w = np.random.normal(size=X.shape[1])
    N = X.shape[1]

    while dw >= epsilon:
        new_w = 2*theta/N*(X.T@X@w - X.T@y)
        dw = np.linalg.norm(w - new_w)
        w = new_w

    return w

# остальные функции
def R2(x, y):
    return 1 - np.sum(np.power(y - x, 2)) / np.sum(np.power(y - y.mean(), 2))

def reg_prediction(X, w):
    return X@w


def RMSE(x, y):
    return np.sqrt(np.sum(np.power(y - x, 2)) / y.shape[0])



# делим на фолды и мутим свое грязное дело
# вторые копии для наглядости
folds_index = 5
fold_size = round(df.shape[0] / folds_index)

features = pd.DataFrame()
RMSE_test = []
RMSE_train = []
R2_test = []
R2_train = []

features2 = pd.DataFrame()
RMSE_test2 = []
RMSE_train2 = []
R2_test2 = []
R2_train2 = []

for i in range(folds_index):
    test = df[i * fold_size:(i + 1) * fold_size]
    if i == 0:
        train = df[(i + 1) * fold_size:]
    else:
        train = df[:i * fold_size]
        if i != 4:
            train = train.append(df[(i + 1) * fold_size:], ignore_index=False)


    Features = train.drop('Target', axis=1)
    Target = train['Target']
    w2 = gradient_descent(Features, Target, 1e-5)
    features2 = features2.append(w2, ignore_index=True)


    train_pred2 = reg_prediction(train.drop('Target', axis=1), w2)
    R2_train2.append(R2(train_pred2, train['Target']))
    RMSE_train2.append(RMSE(train_pred2, train['Target']))

    test_pred2 = reg_prediction(test.drop('Target', axis=1), w2)
    R2_test2.append(R2(test_pred2, test['Target']))
    RMSE_test2.append(RMSE(test_pred2, test['Target']))

# вывод результатов с шагом 0.00001
res_df = pd.DataFrame(np.vstack([R2_test2, R2_train2, RMSE_test2, RMSE_train2]),
                      index=['R2_test', 'R2_train', 'RMSE_test', 'RMSE_train'])
res_df = res_df.append(features2.T)
res_df.columns = ['T1', 'T2', 'T3', 'T4', 'T5']
res_df = pd.concat([res_df, res_df.mean(axis=1).rename('E(mean)'), res_df.std(axis=1).rename('STD')], axis=1)

res_df
# res_df.to_csv('out.csv', sep='\t', encoding='utf-8')


,T1,T2,T3,T4,T5,E(mean),STD
R2_test,-8.130098e-02,-7.088753e-02,-0.076854,-0.092949,-0.094037,-8.320593e-02,0.010099
R2_train,-8.108891e-02,-8.608457e-02,-0.082270,-0.080839,-0.083472,-8.275089e-02,0.002138
RMSE_test,1.064291e+00,1.118804e+00,1.091145,0.970743,0.944462,1.037889e+00,0.076351
RMSE_train,1.033515e+00,1.019902e+00,1.026477,1.057367,1.064625,1.040377e+00,0.019597
Base Time,2.743472e-03,2.816042e-03,0.002818,0.002837,0.002888,2.820546e-03,0.000052
CC1,-4.841268e-03,-4.795421e-03,-0.004665,-0.004993,-0.005002,-4.859320e-03,0.000142
CC2,-7.189701e-03,-7.156220e-03,-0.006993,-0.007298,-0.007476,-7.222476e-03,0.000179
CC3,-1.406478e-03,-1.346789e-03,-0.001295,-0.001529,-0.001361,-1.387467e-03,0.000089
CC4,-5.026728e-03,-4.987072e-03,-0.004856,-0.005179,-0.005200,-5.049711e-03,0.000142
CC5,-4.825007e-03,-4.842632e-03,-0.004748,-0.004821,-0.005094,-4.866183e-03,0.000132
